# 5 percent data training with R->T->R->sigmoid activation for 1000 epochs with dynamic variation in learning rate

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
import os


In [4]:
df = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May6rd5percent_RLV_data 100 points2 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
# df3=pd.DataFrame(df.iloc[:,0:3])
input = df.values
output = df.values 
X = input[:,0:3]
y = output[:,7:9]
# Individual Data
h = X[:,0]
v = X[:,1]
s = X[:,2]
# t = X[:,3]
T = y[:,0]
b = y[:,1]

In [5]:
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2022-05-11 08:45:19.791638: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-05-11 08:45:19.791686: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


import seaborn as sns

import matplotlib.pyplot as plt

In [7]:
def create_model(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(3,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'tanh')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

In [8]:
for i in range(4,11,1):
  model = create_model(2**i)
  opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
  loss = tf.keras.losses.mean_squared_error
  def lr_sch(epoch,lr):
    if (epoch>0) & (epoch<200):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return lr*np.exp(-0.45)
      else:
        return lr
    elif(epoch>200):
        if epoch%50==0:
            return lr*np.exp(-0.225);
        else:
            return lr
    else:
        return lr
  print("The Number of Hidden units used is: ",2**i)
  lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch,verbose = 1)
    
  batchsize = 128;
  epoch = 1000;
  checkpoint_path = f"/home/saichaitanya/Chaitanya/Trained models /5percent data/{2**i}/cp.ckpt"
  # "/content/gdrive/My Drive/May3rd data model/ with hidden units",2**i,"/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)

  # Create a callback that saves the model's weights
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)

  # This may generate warnings related to saving the state of the optimizer.
  # These warnings (and similar warnings throughout this notebook)
  # are in place to discourage outdated usage, and can be ignored.

  model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
  # model.load_weights(checkpoint_path)
  model.fit(X_train,y_train,epochs = 1000,batch_size = 128,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback])
  model.evaluate(X_test,y_test)

The Number of Hidden units used is:  16

Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/1000


2022-05-11 08:45:35.814920: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1380/1381 [============================>.] - ETA: 0s - loss: 0.0250 - accuracy: 0.7833
Epoch 1: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/16/cp.ckpt
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0250 - accuracy: 0.7833 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/1000
1359/1381 [============================>.] - ETA: 0s - loss: 0.0183 - accuracy: 0.8022
Epoch 2: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0183 - accuracy: 0.8021 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/1000
1356/1381 [============================>.] - ETA: 0s - loss: 0.0166 - accuracy: 0.8062
Epoch 3: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - l


Epoch 23: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 23/1000
1368/1381 [============================>.] - ETA: 0s - loss: 0.0054 - accuracy: 0.9112
Epoch 23: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0053 - accuracy: 0.9113 - lr: 0.0010

Epoch 24: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 24/1000
1376/1381 [============================>.] - ETA: 0s - loss: 0.0053 - accuracy: 0.9127
Epoch 24: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0053 - accuracy: 0.9127 - lr: 0.0010

Epoch 25: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 25/1000
1370/1381 [============================>.] - ETA: 0s - loss: 0.0053 - accuracy: 0.9139
Epoch 25: saving model to /home/saichaitanya/Chaitanya

1368/1381 [============================>.] - ETA: 0s - loss: 0.0046 - accuracy: 0.9273
Epoch 44: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0046 - accuracy: 0.9274 - lr: 0.0010

Epoch 45: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 45/1000
1367/1381 [============================>.] - ETA: 0s - loss: 0.0046 - accuracy: 0.9290
Epoch 45: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0046 - accuracy: 0.9290 - lr: 0.0010

Epoch 46: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 46/1000
1378/1381 [============================>.] - ETA: 0s - loss: 0.0045 - accuracy: 0.9309
Epoch 46: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/s

...

Epoch 906/1000
1379/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9721
Epoch 906: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 0.0014 - accuracy: 0.9721 - lr: 1.1109e-05

Epoch 907: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 907/1000
1378/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9719
Epoch 907: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 0.0014 - accuracy: 0.9719 - lr: 1.1109e-05

Epoch 908: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 908/1000
1378/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9718
Epoch 908: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==

Epoch 927/1000
1378/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9720
Epoch 927: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 19s 14ms/step - loss: 0.0014 - accuracy: 0.9720 - lr: 1.1109e-05

Epoch 928: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 928/1000
1380/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9722
Epoch 928: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 19s 14ms/step - loss: 0.0014 - accuracy: 0.9722 - lr: 1.1109e-05

Epoch 929: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 929/1000
1381/1381 [==============================] - ETA: 0s - loss: 0.0014 - accuracy: 0.9720
Epoch 929: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==

Epoch 948/1000
1380/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9720
Epoch 948: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 18s 13ms/step - loss: 0.0014 - accuracy: 0.9720 - lr: 1.1109e-05

Epoch 949: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 949/1000
1378/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9719
Epoch 949: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 18s 13ms/step - loss: 0.0014 - accuracy: 0.9719 - lr: 1.1109e-05

Epoch 950: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 950/1000
1380/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9719
Epoch 950: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==

Epoch 969/1000
1380/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9718
Epoch 969: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 19s 14ms/step - loss: 0.0014 - accuracy: 0.9718 - lr: 8.8707e-06

Epoch 970: LearningRateScheduler setting learning rate to 8.870713827491272e-06.
Epoch 970/1000
1378/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9722
Epoch 970: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 0.0014 - accuracy: 0.9722 - lr: 8.8707e-06

Epoch 971: LearningRateScheduler setting learning rate to 8.870713827491272e-06.
Epoch 971/1000
1380/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9720
Epoch 971: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==

Epoch 990/1000
1381/1381 [==============================] - ETA: 0s - loss: 0.0014 - accuracy: 0.9721
Epoch 990: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 19s 14ms/step - loss: 0.0014 - accuracy: 0.9721 - lr: 8.8707e-06

Epoch 991: LearningRateScheduler setting learning rate to 8.870713827491272e-06.
Epoch 991/1000
1381/1381 [==============================] - ETA: 0s - loss: 0.0014 - accuracy: 0.9721
Epoch 991: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==============================] - 19s 14ms/step - loss: 0.0014 - accuracy: 0.9721 - lr: 8.8707e-06

Epoch 992: LearningRateScheduler setting learning rate to 8.870713827491272e-06.
Epoch 992/1000
1379/1381 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9720
Epoch 992: saving model to /home/saichaitanya/Chaitanya/Trained models /5percent data/1024/cp.ckpt
1381/1381 [==

## Model validation and plotting


In [18]:
i = 10
model = create_model(2**i)
model.load_weights(f"/home/saichaitanya/Chaitanya/Trained models /5percent data/{2**i}/cp.ckpt")
model.predict(X_test)

array([[0.04440644, 0.4717508 ],
       [0.77101207, 0.4773344 ],
       [0.11630222, 0.66566813],
       ...,
       [0.9589534 , 0.4607002 ],
       [0.89817065, 0.45483723],
       [0.15318409, 0.4592329 ]], dtype=float32)

In [13]:
y_test

array([[0.06554793, 0.48950587],
       [0.77689939, 0.47137161],
       [0.13451512, 0.6880959 ],
       ...,
       [0.96224265, 0.45806057],
       [0.87244465, 0.47643685],
       [0.1828455 , 0.45690801]])